In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing packages if needed

In [ ]:
!pip install keras

# Importing packages

In [ ]:
import os
import numpy as np
import pandas as pd
import random as rn

import tensorflow as tf
import keras

from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.models import Sequential

from tensorflow.keras.models import Model, load_model,Sequential
from tensorflow.keras.layers import Input, Dense,Conv2D,MaxPool2D,Activation,Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers

# Setting up paths

In [ ]:
dir_path = '/kaggle/input/rvlcdip/'
os.listdir(dir_path)

In [ ]:
traindf = pd.read_csv('/kaggle/input/rvlcdip/labels_final.csv')
print(traindf.head())
print(traindf.shape)

# Loading and Visualising the Images

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load and display a .tif image
image_path = '/kaggle/input/rvlcdip/data_final/imagesv/v/o/h/voh71d00/509132755+-2755.tif'
image = Image.open(image_path)

plt.imshow(image)
plt.axis('off')  # Turn off axis labels
plt.show()

# Chekcing how many class images are present

In [ ]:
traindf["label"].value_counts()

In [ ]:
traindf.head()

# Initializing ImageDataGenerator with Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

# Setting up train Generator

In [ ]:
%%time

# datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator = datagen.flow_from_dataframe(dataframe= traindf,
                                              directory='/kaggle/input/rvlcdip/data_final/',
                                              x_col = 'path',
                                              y_col = 'label',
                                              subset = 'training',
                                              batch_size=100,
                                              seed=42,
                                              shuffle=True,
                                              class_mode="raw",
                                              target_size=(224,224))



# Setting up Validation Generator

In [ ]:
valid_generator = datagen.flow_from_dataframe(dataframe= traindf,
                                              directory='/kaggle/input/rvlcdip/data_final/',
                                              x_col = 'path',
                                              y_col = 'label',
                                              subset = 'validation',
                                              batch_size=100,
                                              seed=42,
                                              shuffle=True,
                                              class_mode="raw",
                                              target_size=(224,224))

# Creating directories to save models

In [ ]:
!mkdir model_save1 logs1

# Model-1 Custom CNN from Scratch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')  # Assuming 16 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Printing model summary

In [ ]:
model.summary()

# Initialisng all the callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os
import datetime

checkpoint_path = "/kaggle/working/model_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.keras"
log_dir = "/kaggle/working/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

# Model training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
import os
import datetime

checkpoint_path = "/kaggle/working/model_checkpoints/weights.{epoch:02d}-{val_loss:.2f}.keras"
log_dir = "/kaggle/working/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    TensorBoard(log_dir=log_dir, histogram_freq=1)
]

# Model Evaluaiton

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

# Model Performance Plots

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()